# Lead Field Polar Sensitivity Visualization

This notebook visualizes directional sensitivity (magnitude of lead field vectors) for different electrode/device types at multiple radial distances from a reference electrode. Each CSV file is assumed to contain reduced lead field samples (vector components Ex,Ey,Ez) sampled uniformly over polar angle for concentric rings.

Workflow
1. Load preprocessed per-angle field samples for each device (DISC, Neuropixel/IMEC, SEEG).
2. Convert Cartesian components to magnitude: |E| = sqrt(Ex^2 + Ey^2 + Ez^2).
3. Slice magnitudes into blocks (33 samples per radius) → eight radii.
4. Optionally apply circular smoothing (wrap convolution) to reduce angular noise.
5. Plot selected radius in a polar chart and save figure.

File Expectations
- CSV layout concatenates radii sequentially; each block of 33 rows corresponds to a radius.
- Angle step assumed 360 / 32 = 11.25 degrees plus closing sample at 360.

Customize
- Change `folder` to point at your unzipped dataset root.
- Replace CSV filenames as needed.
- Toggle which radius to plot (see final plotting cell).
- Enable smoothing for aesthetically cleaner curves.

Outputs
- PNG polar plots written into `outputs/` subfolder constructed from `folder`.

Next Extensions
- Normalize magnitudes across devices for relative comparison.
- Overlay confidence bands if variability data available.
- Export processed arrays for reuse.


# Polar Plot of Lead Fields
Visualizes directionality for different devices at given radii from electrodes.

Use this notebook after generating reduced lead field slices. For full-volume fields, preprocess to polar slices externally (not provided here).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from os import path
import os

# ------------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------------
# Root dataset folder (UNZIPPED). Update this path before running.
folder = r"...\SEPIO_dataset"  # Example: r"C:\Data\SEPIO_dataset"
output = path.join(folder, 'outputs')
os.makedirs(output, exist_ok=True)

# Input CSVs: each expected to contain concatenated angle samples per radius.
disc_fields_file = path.join(folder, 'polar_plots', 'test_disc.csv')
imec_fields_file = path.join(folder, 'polar_plots', 'test_imec.csv')
seeg_fields_file = path.join(folder, 'polar_plots', 'test_seeg.csv')

# Validation warnings (non-fatal)
for f in [disc_fields_file, imec_fields_file, seeg_fields_file]:
    if not path.isfile(f):
        print(f"[WARN] Missing CSV file: {f}")


In [ ]:
# Load reduced lead fields and separate by radial distance
fields_disc = np.genfromtxt(disc_fields_file)
fields_disc = np.nan_to_num(fields_disc)
fields_imec = np.genfromtxt(imec_fields_file)
fields_imec = np.nan_to_num(fields_imec)
fields_seeg = np.genfromtxt(seeg_fields_file)
fields_seeg = np.nan_to_num(fields_seeg)


fields_mag = np.sqrt(np.add(np.square(fields_disc[:,0]),np.square(fields_disc[:,1]),np.square(fields_disc[:,2])))
angles = np.arange(0, 361, 11.25)
angles = np.deg2rad(angles)
mags1 = fields_mag[0:33]
mags2 = fields_mag[33:66]
mags3 = fields_mag[66:99]
mags4 = fields_mag[99:132]
mags5 = fields_mag[132:165]
mags6 = fields_mag[165:198]
mags7 = fields_mag[198:231]
mags8 = fields_mag[231:264]

disc2_mag = np.sqrt(np.add(np.square(fields_disc[:,0]),np.square(fields_disc[:,1]),np.square(fields_disc[:,2])))
angles2 = angles
disc21 = disc2_mag[0:33]
disc22 = disc2_mag[33:66]
disc23 = disc2_mag[66:99]
disc24 = disc2_mag[99:132]
disc25 = disc2_mag[132:165]
disc26 = disc2_mag[165:198]
disc27 = disc2_mag[198:231]
disc28 = disc2_mag[231:264]

imec_mag = np.sqrt(np.add(np.square(fields_imec[:,0]),np.square(fields_imec[:,1]),np.square(fields_imec[:,2])))
angles_imec = np.add(angles, np.pi*-0.5)
imec1 = imec_mag[0:33]
imec2 = imec_mag[33:66]
imec3 = imec_mag[66:99]
imec4 = imec_mag[99:132]
imec5 = imec_mag[132:165]
imec6 = imec_mag[165:198]
imec7 = imec_mag[198:231]
imec8 = imec_mag[231:264]

seeg_mag = np.sqrt(np.add(np.square(fields_seeg[:,0]),np.square(fields_seeg[:,1]),np.square(fields_seeg[:,2])))
angles_seeg = angles
seeg1 = seeg_mag[0:33]
seeg2 = seeg_mag[33:66]
seeg3 = seeg_mag[66:99]
seeg4 = seeg_mag[99:132]
seeg5 = seeg_mag[132:165]
seeg6 = seeg_mag[165:198]
seeg7 = seeg_mag[198:231]
seeg8 = seeg_mag[231:264]

In [ ]:
### Optional smoothing
window_size = 3
m = 'valid'
pad = (window_size-1)//2
mags7 = np.pad(mags7, (pad,pad), mode='wrap')
imec7 = np.pad(imec7, (pad,pad), mode='wrap')
seeg7 = np.pad(seeg7, (pad,pad), mode='wrap')
mags7 = np.convolve(mags7, np.ones(window_size)/window_size, mode=m)
imec7 = np.convolve(imec7, np.ones(window_size)/window_size, mode=m)
seeg7 = np.convolve(seeg7, np.ones(window_size)/window_size, mode=m)
mags8 = np.pad(mags8, (pad,pad), mode='wrap')
imec8 = np.pad(imec8, (pad,pad), mode='wrap')
seeg8 = np.pad(seeg8, (pad,pad), mode='wrap')
mags8 = np.convolve(mags8, np.ones(window_size)/window_size, mode=m)
imec8 = np.convolve(imec8, np.ones(window_size)/window_size, mode=m)
seeg8 = np.convolve(seeg8, np.ones(window_size)/window_size, mode=m)

In [ ]:
### Plot the polar sensitivity
# Choose a radius below

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

# 800 um range
if True:
    radius = '800um'
    ax.plot(angles, mags2, label='DISC')
    ax.plot(angles_imec, imec2, label='Neuropixel')
    ax.plot(angles_seeg, seeg2, label='SEEG')

# 1.5 mm range
if False:
    radius = '1.5mm'
    ax.plot(angles, mags3, label='DISC')
    ax.plot(angles_imec, imec3, label='Neuropixel')
    ax.plot(angles_seeg, seeg3, label='SEEG')

# 3.5 mm range
if False:
    radius = '3.5mm'
    ax.plot(angles, mags4, label='DISC')
    ax.plot(angles_imec, imec4, label='Neuropixel')
    ax.plot(angles_seeg, seeg4, label='SEEG')

# Additional options for various radii

# ax.plot(angles, mags1, label='DISC')
# ax.plot(angles, mags5, label='DISC')
# ax.plot(angles, mags6, label='DISC')
# ax.plot(angles, mags7, label='DISC')
# ax.plot(angles, mags8, label='DISC')

# ax.plot(angles_imec, imec1, label='Neuropixel')
# ax.plot(angles_imec, imec5, label='Neuropixel')
# ax.plot(angles_imec, imec6, label='Neuropixel')
# ax.plot(angles_imec, imec7, label='Neuropixel')
# ax.plot(angles_imec, imec8, label='Neuropixel')

# ax.plot(angles_seeg, seeg1, label='SEEG')
# ax.plot(angles_seeg, seeg5, label='SEEG')
# ax.plot(angles_seeg, seeg6, label='SEEG')
# ax.plot(angles_seeg, seeg7, label='SEEG')
# ax.plot(angles_seeg, seeg8, label='SEEG')

# Optional X,Y,Z components on DiSc LF
if False:
    ax.plot(angles, np.abs(fields_disc[33:66, 0]), label='X', color='red')
    ax.plot(angles, np.abs(fields_disc[33:66, 1]), label='Y', color='blue')
    ax.plot(angles, np.abs(fields_disc[33:66, 2]), label='Z', color='green')

ax.legend()
ax.grid(True)

ax.set_title(f"LF Magnitudes at {radius} radius", va='bottom')
plt.savefig(path.join(output,f"2_polar_plot-{radius}.png"), format='png',dpi=300)
plt.show()